# 🎨 CNN-Based Image Watermarking using DWT
## Kaggle - Optimized Version

This notebook trains and evaluates a deep learning watermarking system.

**Features:**
- Invisible watermark embedding
- Robust against 7 attack types
- Automatic evaluation with metrics
- GPU accelerated training

**Kaggle Setup:**
- Enable GPU: Settings → Accelerator → GPU T4 x2
- Internet: Settings → Internet → ON

---

## 📦 Step 1: Setup Environment

In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", "✓ YES" if tf.config.list_physical_devices('GPU') else "✗ NO")
print("\n⚠️ If GPU is not available, go to: Settings → Accelerator → GPU T4 x2")

In [ ]:
# Clone repository
import os

# Kaggle working directory is /kaggle/working
os.chdir('/kaggle/working')

# Clean up any existing directory
if os.path.exists('Watermarking-cnn'):
    print("Removing existing directory...")
    !rm -rf Watermarking-cnn

# Clone fresh
print("Cloning repository...")
!git clone https://github.com/Mehulsri07/Watermarking-cnn.git

# Change to project directory
os.chdir('Watermarking-cnn')

# Verify location
print(f"\n📁 Current directory: {os.getcwd()}")
print(f"📂 Contents: {os.listdir('.')[:10]}...")

In [ ]:
# Install dependencies with NumPy fix for Kaggle
print("Installing dependencies...")
!pip install -q 'numpy<2.0' tensorflow-wavelets tensorflow-addons opencv-python scikit-image 'matplotlib>=3.8.0'
print("✓ Dependencies installed")

# Verify NumPy version
import numpy as np
print(f"NumPy version: {np.__version__}")

## 📥 Step 2: Download Sample Images

In [ ]:
# Download sample images from Lorem Picsum
import urllib.request
import os

def download_samples(num_train=20, num_test=5):
    os.makedirs('train_images', exist_ok=True)
    os.makedirs('test_images', exist_ok=True)
    
    print(f"Downloading {num_train} training images...")
    for i in range(num_train):
        try:
            url = f"https://picsum.photos/256/256?random={i}"
            urllib.request.urlretrieve(url, f"train_images/train_{i:03d}.jpg")
            if (i+1) % 10 == 0:
                print(f"  Downloaded {i+1}/{num_train}")
        except Exception as e:
            print(f"  Error downloading image {i}: {e}")
    
    print(f"\nDownloading {num_test} test images...")
    for i in range(num_test):
        try:
            url = f"https://picsum.photos/256/256?random={100+i}"
            urllib.request.urlretrieve(url, f"test_images/test_{i:03d}.jpg")
            if (i+1) % 5 == 0:
                print(f"  Downloaded {i+1}/{num_test}")
        except Exception as e:
            print(f"  Error downloading image {i}: {e}")
    
    print(f"\n✓ Downloaded {num_train} training + {num_test} test images")

download_samples(num_train=20, num_test=5)

## ⚙️ Step 3: Configure Training

In [ ]:
# Training configuration - Optimized for Kaggle
EPOCHS = 10          # Number of training epochs
BATCH_SIZE = 2       # Batch size
LEARNING_RATE = 0.001

print("Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Training Images: 20")
print(f"  Test Images: 5")
print("\n⏱️ Expected training time: ~5-10 minutes on Kaggle GPU")

## 🚀 Step 4: Train Model

In [ ]:
# Verify setup before training
import os
import sys

print("Pre-training checks:")
print(f"Current directory: {os.getcwd()}")

# Check directories
print(f"\nChecking required directories:")
for dir_name in ['models', 'attacks', 'data_loaders', 'utils']:
    exists = os.path.exists(dir_name)
    print(f"  {'✓' if exists else '✗'} {dir_name}/")
    if exists:
        init_exists = os.path.exists(os.path.join(dir_name, '__init__.py'))
        print(f"    {'✓' if init_exists else '✗'} __init__.py")

# Check images
print(f"\nChecking images:")
train_count = len([f for f in os.listdir('train_images') if f.endswith(('.jpg', '.png'))]) if os.path.exists('train_images') else 0
test_count = len([f for f in os.listdir('test_images') if f.endswith(('.jpg', '.png'))]) if os.path.exists('test_images') else 0
print(f"  Training images: {train_count}")
print(f"  Test images: {test_count}")

# Test imports
print(f"\nTesting imports:")
try:
    from models.wavetf_model import WaveTFModel
    print("  ✓ models.wavetf_model")
except Exception as e:
    print(f"  ✗ models.wavetf_model: {e}")

try:
    from data_loaders.merged_data_loader import MergedDataLoader
    print("  ✓ data_loaders.merged_data_loader")
except Exception as e:
    print(f"  ✗ data_loaders.merged_data_loader: {e}")

print("\n" + "="*60)
if train_count == 0 or test_count == 0:
    print("⚠️ Warning: No images found! Run Step 2 first.")
else:
    print("✓ Ready to train!")
print("="*60)

In [ ]:
# Run training and evaluation
!python train_and_evaluate.py

## 📊 Step 5: View Results

In [ ]:
# Check if training completed successfully
import os

print("Checking training results...\n")

checks = {
    'Model weights': 'config_1_baseline/final_model_weights.h5',
    'Evaluation report': 'config_1_baseline/evaluation_results/evaluation_report.json',
    'Summary chart': 'config_1_baseline/evaluation_results/summary_metrics.png',
    'Result images': 'config_1_baseline/evaluation_results/images/'
}

all_good = True
for name, path in checks.items():
    if os.path.exists(path):
        print(f"✓ {name}: Found")
    else:
        print(f"✗ {name}: Missing")
        all_good = False

print("\n" + "="*60)
if all_good:
    print("✅ Training completed successfully!")
    print("You can now view the results below.")
else:
    print("⚠️ Training may not have completed.")
    print("Please run the training cell (Step 4) first.")
print("="*60)

In [ ]:
# Display summary metrics
from IPython.display import Image, display
import json
import os

results_path = 'config_1_baseline/evaluation_results/summary_metrics.png'
if os.path.exists(results_path):
    print("📊 Performance Summary:")
    display(Image(results_path))
else:
    print("⚠️ Results not found. Make sure training completed successfully.")
    print("Run the training cell above first!")

In [ ]:
# Show detailed results
import os
import json

report_path = 'config_1_baseline/evaluation_results/evaluation_report.json'
if os.path.exists(report_path):
    with open(report_path, 'r') as f:
        report = json.load(f)
    
    print("\n📋 Detailed Results:\n")
    print("="*80)
    for attack_name, stats in report['attack_statistics'].items():
        print(f"\n{attack_name}:")
        print(f"  PSNR: {stats['avg_psnr']:.2f} dB")
        print(f"  SSIM: {stats['avg_ssim']:.4f}")
        print(f"  BER:  {stats['avg_ber']:.2f}%")
    print("\n" + "="*80)
else:
    print("⚠️ Evaluation report not found.")
    print("Make sure training completed successfully!")

In [ ]:
# Display sample visualizations
import glob
import os
from IPython.display import Image, display

images_dir = 'config_1_baseline/evaluation_results/images/'
if os.path.exists(images_dir):
    print("\n🖼️ Sample Visualizations:\n")
    image_files = glob.glob(images_dir + '*.png')[:3]
    
    if image_files:
        for img_file in image_files:
            print(f"\n{os.path.basename(img_file)}:")
            display(Image(img_file, width=800))
    else:
        print("No visualization images found.")
else:
    print("⚠️ Results directory not found.")
    print("Make sure training completed successfully!")

## 💾 Step 6: Save Results to Kaggle Output

In [ ]:
# Copy results to Kaggle output directory
# Files in /kaggle/working are saved automatically
import shutil
import os

if os.path.exists('config_1_baseline'):
    print("Copying results to Kaggle output...")
    
    # Copy model weights
    if os.path.exists('config_1_baseline/final_model_weights.h5'):
        shutil.copy('config_1_baseline/final_model_weights.h5', '/kaggle/working/')
        print("✓ Model weights saved")
    
    # Copy evaluation report
    if os.path.exists('config_1_baseline/evaluation_results/evaluation_report.json'):
        shutil.copy('config_1_baseline/evaluation_results/evaluation_report.json', '/kaggle/working/')
        print("✓ Evaluation report saved")
    
    # Copy summary chart
    if os.path.exists('config_1_baseline/evaluation_results/summary_metrics.png'):
        shutil.copy('config_1_baseline/evaluation_results/summary_metrics.png', '/kaggle/working/')
        print("✓ Summary chart saved")
    
    print("\n✓ Results saved to Kaggle output!")
    print("You can download them from the Output tab.")
else:
    print("⚠️ No results to save. Run training first.")

## 🧪 Step 7: Test Custom Image (Optional)

In [ ]:
# Test with a custom image
import cv2
import numpy as np
from IPython.display import Image as IPImage, display
import os

# Use one of the test images
test_image_path = 'test_images/test_000.jpg'

if os.path.exists(test_image_path):
    print(f"Testing with: {test_image_path}\n")
    
    # Load image
    img = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)
    
    # Generate watermark
    watermark = np.random.randint(0, 2, size=(256,)).astype(np.float32)
    
    # Load model
    from models.wavetf_model import WaveTFModel
    wavetf_model = WaveTFModel(image_size=(256, 256, 1), watermark_size=(256,))
    model = wavetf_model.get_model()
    model.load_weights('config_1_baseline/final_model_weights.h5')
    
    # Embed watermark
    img_batch = np.expand_dims(img, axis=0)
    wm_batch = np.expand_dims(watermark, axis=0)
    attack_batch = np.array([[0]], dtype=np.int32)
    
    watermarked, extracted = model.predict([img_batch, wm_batch, attack_batch])
    
    # Save result
    cv2.imwrite('watermarked_output.png', (watermarked[0].squeeze() * 255).astype(np.uint8))
    
    print("📸 Results:")
    display(IPImage('watermarked_output.png'))
    
    # Calculate metrics
    from utils.metrics import calculate_psnr, calculate_ssim, calculate_ber
    psnr = calculate_psnr(img_batch, watermarked)
    ssim = calculate_ssim(img_batch, watermarked)
    ber = calculate_ber(wm_batch, extracted)
    
    print(f"\n📊 Metrics:")
    print(f"  PSNR: {psnr:.2f} dB")
    print(f"  SSIM: {ssim:.4f}")
    print(f"  BER:  {ber:.2f}%")
else:
    print("⚠️ Test image not found. Run Step 2 first.")

---
## ✅ Done!

Your watermarking model is trained and evaluated!

**Results saved to:**
- `/kaggle/working/final_model_weights.h5` - Model weights
- `/kaggle/working/evaluation_report.json` - Detailed metrics
- `/kaggle/working/summary_metrics.png` - Performance chart

**Download from:** Output tab (top right)

**Next Steps:**
- Increase EPOCHS for better accuracy
- Add more training images
- Test with your own images
- Share your notebook!

**Questions?** Open an issue on GitHub!